# Semantic Segmentation with convpaint and DINOv2
<a target="_blank" href="https://colab.research.google.com/github/quasar1357/dino_paint/blob/main/dino_paint.ipynb?hl=de"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebooks demonstrates how to run a semantic segmentation on an image using DINOv2 for feature extraction and a random forest algorithm for classification. It is based on the notebook provided by convpaint.


## Installs and Imports

In [ ]:
if False:
    !pip install -r https://raw.githubusercontent.com/quasar1357/dino_paint/main/requirements.txt

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from matplotlib import pyplot as plt
import skimage
from dino_paint_utils import (train_dino_forest,
                              predict_dino_forest,
                              selfpredict_dino_forest,
                              test_dino_forest,
                              summarize_results)

show_napari = True

## Choose the model

1) Choose the **DINOv2 model** to be used (assign None to not use DINOv2):

|key | model| features
|---|---|---|
|'s' | dinov2_vits14| 384|
|'b' | dinov2_vitb14| 768|
|'l' | dinov2_vitl14| 1024|
|'g' | dinov2_vitg14| 1536|
|+ '_r' | *base_model*_reg (not supported yet)| add registers|

2) Choose the **layers of DINOv2** to used features (give a list of indices 0-11); each layer has the number of features specific for the model as listed in the table above.

3) Choose the **extra pads** used before feature extraction with DINOv2, leading to a shift of the patches used in the feature extraction.

4) Choose the **layers of VGG16** to be attatched as additional features (give a list of indices; only use Conv2d layers; assign None to not use VGG16):

|index|layer|
|---|---|
|**0**|**Conv2d3, 64, kernel_size=3, stride=1, padding=1**|
|1|ReLUinplace=True|
|**2**|**Conv2d64, 64, kernel_size=3, stride=1, padding=1**|
|3|ReLUinplace=True|
|4|MaxPool2dkernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False|
|**5**|**Conv2d64, 128, kernel_size=3, stride=1, padding=1**|
|6|ReLUinplace=True|
|**7**|**Conv2d128, 128, kernel_size=3, stride=1, padding=1**|
|8|ReLUinplace=True|
|9|MaxPool2dkernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False|
|**10**|**Conv2d128, 256, kernel_size=3, stride=1, padding=1**|
|11|ReLUinplace=True|
|**12**|**Conv2d256, 256, kernel_size=3, stride=1, padding=1**|
|13|ReLUinplace=True|
|**14**|**Conv2d256, 256, kernel_size=3, stride=1, padding=1**|
|15|ReLUinplace=True|
|16|MaxPool2dkernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False|
|**17**|**Conv2d256, 512, kernel_size=3, stride=1, padding=1**|
|18|ReLUinplace=True|
|**19**|**Conv2d512, 512, kernel_size=3, stride=1, padding=1**|
|20|ReLUinplace=True|
|**21**|**Conv2d512, 512, kernel_size=3, stride=1, padding=1**|
|22|ReLUinplace=True|
|23|MaxPool2dkernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False|
|**24**|**Conv2d512, 512, kernel_size=3, stride=1, padding=1**|
|25|ReLUinplace=True|
|**26**|**Conv2d512, 512, kernel_size=3, stride=1, padding=1**|
|27|ReLUinplace=True|
|**28**|**Conv2d512, 512, kernel_size=3, stride=1, padding=1**|
|29|ReLUinplace=True|
|30|MaxPool2dkernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False|

5) Choose if the **image itself** (3 rgb channels) shall be added as features.

6) Choose the **scale factor combinations** to use, as a tuple (DINOv2_scales, VGG16_scales), whereas DINOv2_scales and VGG16_scales each is a list of scales to use for the respective model.



In [3]:
dinov2_model = 's'
dinov2_layers = [8,9,10,11]
extra_pads= () #[2,4,6,8,10,12]
vgg16 = None #[2,7] # [0,2,5,7,10,12,14,17,19,21,24,26,28]
image_as_feature = False#True
scales = {"DINOv2": [1.0, 1.32, 1.73, 2.28, 3.0][:1],
          "VGG16": (1,2)}

upscale_order = 0
pad_mode = 'reflect'

## Online loading & displaying

Define a function for loading images from url.

In [4]:
import PIL
import urllib

def load_url_to_array(url, from_github=False):
    return np.array(PIL.Image.open(urllib.request.urlopen(['', 'https://github.com/quasar1357/dino_paint/raw/main/images_and_labels/'][from_github] + url)))

Disable showing results in napari if working online (in colab), since it is not supported.

In [5]:
show_napari = False

## Train

Load an image and its annotation/labels to train the model on.

In [3]:
from napari_convpaint.convpaint_sample import create_annotation_cell3d
image_to_train = skimage.data.cells3d()[30, 1]
labels_to_train = create_annotation_cell3d()[0][0]
image_to_train = image_to_train[:, :126]
labels_to_train = labels_to_train[:, :126]

# LOAD ASTRONAUT IMAGE (RGB) AND ANNOTATION
# image_to_train = skimage.data.astronaut()#[0:504,0:504,:]
# labels_to_train = load_url_to_array('astro_labels.tif', True)#[0:504,0:504]

# LOAD HARDER CELL IMAGE AND ITS LABELS
# image_to_train = load_url_to_array('00_00016.tiff', True)
# labels_to_train = load_url_to_array('00_00016_labels.tiff', True)

Exctract the features using DINOv2 and/or VGG16 and use them to train a random forest classifier.

In [ ]:
train = train_dino_forest(image_to_train, labels_to_train,
                          upscale_order=upscale_order, pad_mode=pad_mode, extra_pads=extra_pads, scales=scales,
                          dinov2_model=dinov2_model, dinov2_layers=dinov2_layers, vgg16_layers=vgg16, append_image_as_feature=image_as_feature,
                          show_napari=False)
random_forest, image_train, labels_train, feature_space_train = train

## Predict

Load an image to predict the labels for using the trained model above.

In [ ]:
image_to_pred = skimage.data.cells3d()
image_to_pred = image_to_pred[40, 1][:,125:251]
ground_truth = load_url_to_array('cells_cross_ground_truth.tif', True)

# LOAD AN IMAGE TO PREDICT BASED ON THE CLASSIFIER TRAINED ON THE ASTRONAUT IMAGE
# image_to_pred = skimage.data.camera()
# ground_truth = load_url_to_array('cam_ground_truth.tif', True)
# image_to_pred = skimage.data.cat()
# image_to_pred = skimage.data.horse().astype(np.int32)
# ground_truth = None

Exctract the features and use them together with the trained classifier to make a prediciton for the labels.

In [ ]:
pred = predict_dino_forest(image_to_pred, random_forest, ground_truth=ground_truth,
                           upscale_order=upscale_order, pad_mode=pad_mode, extra_pads=extra_pads, scales=scales,
                           dinov2_model=dinov2_model, dinov2_layers=dinov2_layers, vgg16_layers=vgg16, append_image_as_feature=image_as_feature,
                           show_napari=show_napari)
predicted_labels, image_pred, feature_space_pred, acc = pred

If can't show in napari, show using matplotlib (excluding the feature space).

In [ ]:
if not show_napari:
    ax = plt.subplots(1, 2)[1]
    ax[0].imshow(predicted_labels)
    ax[0].axis('off')
    ax[0].set_title('Predicted labels')

    ax[1].imshow(image_pred)
    ax[1].axis('off')
    ax[1].set_title('Image')

    plt.show()

## Selfpredict

We can also directly do a training and prediction on the same image (extracting the features only once).

In [4]:
# self_pred_image = skimage.data.astronaut()#[0:504,0:504,:]
# self_pred_labels = load_url_to_array('astro_labels.tif', True)#[0:504,0:504]
# ground_truth = load_url_to_array('astro_ground_truth.tif', True)#[0:504,0:504]

self_pred_image = np.array(PIL.Image.open("images_and_labels/dogs.jpg"))
self_pred_labels = np.array(PIL.Image.open("images_and_labels/dogs_labels.tif"))
ground_truth = None

# self_pred_image = image_to_train
# self_pred_labels = labels_to_train
# # ground_truth = None

self_pred = selfpredict_dino_forest(self_pred_image, self_pred_labels, ground_truth,
                                    upscale_order=upscale_order, pad_mode=pad_mode, extra_pads=extra_pads, scales=scales,
                                    dinov2_model=dinov2_model, dinov2_layers=dinov2_layers, vgg16_layers=vgg16, append_image_as_feature=image_as_feature,
                                    show_napari=show_napari)
predicted_labels, image_scaled, labels_scaled, feature_space, acc = self_pred

C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


If can't show in napari, show using matplotlib (excluding the feature space).

In [ ]:
if not show_napari:
    ax = plt.subplots(1, 3)[1]
    ax[0].imshow(predicted_labels)
    ax[0].axis('off')
    ax[0].set_title('Predicted labels')

    ax[1].imshow(image_scaled)
    ax[1].axis('off')
    ax[1].set_title('Image')

    ax[2].imshow(labels_scaled)
    ax[2].axis('off')
    ax[2].set_title('Labels')

    plt.show()

## Tests against ground truth

In [53]:
# Load/define image
image_to_train = skimage.data.astronaut()#[200:300,250:400]
labels_to_train = load_url_to_array('astro_labels.tif', True)#[200:300,250:400]
image_to_pred = None #skimage.data.camera()
ground_truth = load_url_to_array('astro_ground_truth.tif', True)#[200:300,250:400]

# Create possible combinations of VGG16 layers and scales
all_vggs = [0,2,5,7,10,12,14,17,19,21,24,26,28]
chosen_vggs = [0,2,5,7,10,12,14,17,19,21,24,26,28]
single_vggs = [[i] for i in chosen_vggs]
consecutive_vggs = [chosen_vggs[:s] for s in range(1, 1 + len(chosen_vggs))]
dual_vggs = [[chosen_vggs[i], chosen_vggs[j]] for i in range(len(chosen_vggs)) for j in range(i+1, len(chosen_vggs))]

all_scales = [1.0, 1.32, 1.73, 2.28, 3.0]
chosen_scales = all_scales[:3]
single_scales = [{"DINOv2": [d], "VGG16": [v]} for d in chosen_scales for v in chosen_scales]
consecutive_scales = [{"DINOv2": chosen_scales[:d], "VGG16": chosen_scales[:v]} for d in range(1, 1 + len(chosen_scales)) for v in range(1, 1 + len(chosen_scales))]

# Choose what to use
dino_models = [None, 's']#, 'b']
dino_layer_combos = [(),
                     [8, 9, 10, 11]]
vgg_layer_combos = [None,
                    [2, 7]]#,
                    # [24,26,28]]
scale_combos = [{"DINOv2": (), "VGG16": ()},
                {"DINOv2": (), "VGG16": (1,2)}]

In [64]:
test = test_dino_forest(image_to_train, labels_to_train, ground_truth, image_to_pred,
                        dinov2_models=dino_models, dinov2_layer_combos=dino_layer_combos, vgg16_layer_combos=vgg_layer_combos, scale_combos=scale_combos,
                        write_csv=False)
test

Running tests for DINOv2 model None, layers ()...
    Running tests for VGG16 layers None...
        Running tests with scale combination {'DINOv2': (), 'VGG16': ()}...


d:\Documents\Beruf & Ausbildung\Ausbildung\22-08 MSc BioInf\- Research Project\dino_paint\dino_paint_utils.py:418: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  'VGG16 Layers': vgg16_layers,


        Running tests with scale combination {'DINOv2': (), 'VGG16': (1, 2)}...
    Running tests for VGG16 layers [2, 7]...
        Running tests with scale combination {'DINOv2': (), 'VGG16': ()}...
        Running tests with scale combination {'DINOv2': (), 'VGG16': (1, 2)}...
Running tests for DINOv2 model None, layers [8, 9, 10, 11]...
    Running tests for VGG16 layers None...
        Running tests with scale combination {'DINOv2': (), 'VGG16': ()}...
        Running tests with scale combination {'DINOv2': (), 'VGG16': (1, 2)}...
    Running tests for VGG16 layers [2, 7]...
        Running tests with scale combination {'DINOv2': (), 'VGG16': ()}...
        Running tests with scale combination {'DINOv2': (), 'VGG16': (1, 2)}...
Running tests for DINOv2 model s, layers ()...
    Running tests for VGG16 layers None...
        Running tests with scale combination {'DINOv2': (), 'VGG16': ()}...
        Running tests with scale combination {'DINOv2': (), 'VGG16': (1, 2)}...
    Running

,DINOv2 Model,DINOv2 Layers,DINOv2 Scales,VGG16 Layers,VGG16 Scales,Accuracy,Execution Time
0,None,x_norm_patchtokens,(),None,(),0.806255,3.649115
1,None,x_norm_patchtokens,(),None,"(1, 2)",0.810303,3.616980
2,None,x_norm_patchtokens,(),"[2, 7]",(),0.855434,28.918947
3,None,x_norm_patchtokens,(),"[2, 7]","(1, 2)",0.853756,46.380057
4,None,"[8, 9, 10, 11]",(),None,(),0.806843,2.900054
5,None,"[8, 9, 10, 11]",(),None,"(1, 2)",0.809288,2.834713
6,None,"[8, 9, 10, 11]",(),"[2, 7]",(),0.862934,16.715321
7,None,"[8, 9, 10, 11]",(),"[2, 7]","(1, 2)",0.851521,52.145845
8,s,x_norm_patchtokens,(),None,(),0.963902,10.347939
9,s,x_norm_patchtokens,(),None,"(1, 2)",0.962364,7.850419


In [101]:
for col in test.columns[:-2]:
    print(f"\n", test.groupby(col)[["Accuracy", "Execution Time"]].mean())


               Accuracy  Execution Time
DINOv2 Model                          
None          0.832042       19.645129
s             0.962854       19.346007

                     Accuracy  Execution Time
DINOv2 Layers                               
[8, 9, 10, 11]      0.897601       20.619041
x_norm_patchtokens  0.897295       18.372096

                Accuracy  Execution Time
DINOv2 Scales                          
()             0.897448       19.495568

               Accuracy  Execution Time
VGG16 Layers                          
None          0.885926        7.052550
[2, 7]        0.908970       31.938587

               Accuracy  Execution Time
VGG16 Scales                          
()            0.898166       13.988281
(1, 2)        0.896730       25.002856
